In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch < 15:
        return 0.001
    elif epoch < 30:
        return 0.0005
    elif epoch < 45:
        return 0.0001
    else:
        return 0.00005

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.58653, saving model to best_m.h5
 - 3s - loss: 0.6744 - acc: 0.7275 - val_loss: 0.5865 - val_acc: 0.6534
Epoch 2/100
Epoch 00002: val_loss improved from 0.58653 to 0.47592, saving model to best_m.h5
 - 2s - loss: 0.5395 - acc: 0.7583 - val_loss: 0.4759 - val_acc: 0.7805
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4337 - acc: 0.7950 - val_loss: 0.5093 - val_acc: 0.8055
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.3969 - acc: 0.8231 - val_loss: 0.5487 - val_acc: 0.7681
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.3227 - acc: 0.8516 - val_loss: 0.5301 - val_acc: 0.8229
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.3312 - acc: 0.8616 - val_loss: 1.7482 - val_acc: 0.5586
Epoch 7/100
Epoch 00007: val_loss improved from 0.47592 to 0.26377, saving model to best_m.h5
 - 2s - loss: 0.3667 - acc: 0.8481 - val_loss: 0.2638 - val_acc: 0.8703
Epoch 8/100
Epoch 

Epoch 63/100
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.1447 - acc: 0.9383 - val_loss: 0.1754 - val_acc: 0.9277
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1578 - acc: 0.9375 - val_loss: 0.1756 - val_acc: 0.9277
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1373 - acc: 0.9475 - val_loss: 0.1737 - val_acc: 0.9202
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1459 - acc: 0.9458 - val_loss: 0.1773 - val_acc: 0.9302
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1578 - acc: 0.9417 - val_loss: 0.1743 - val_acc: 0.9252
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1733 - acc: 0.9248 - val_loss: 0.1867 - val_acc: 0.9152
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1301 - acc: 0.9425 - val_loss: 0.1805 - val_acc: 0.9277
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1486 - acc: 0.9406 - val_loss: 0.1760 - val_acc: 0.9127
Epoch 71/100
Epo

Epoch 51/100
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.1500 - acc: 0.9433 - val_loss: 0.2434 - val_acc: 0.9102
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 3s - loss: 0.1736 - acc: 0.9264 - val_loss: 0.2391 - val_acc: 0.9052
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 4s - loss: 0.1263 - acc: 0.9517 - val_loss: 0.2258 - val_acc: 0.9152
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 4s - loss: 0.1651 - acc: 0.9275 - val_loss: 0.2325 - val_acc: 0.9102
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 4s - loss: 0.1686 - acc: 0.9383 - val_loss: 0.2323 - val_acc: 0.9127
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 4s - loss: 0.1668 - acc: 0.9333 - val_loss: 0.2239 - val_acc: 0.8978
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 5s - loss: 0.1471 - acc: 0.9458 - val_loss: 0.2674 - val_acc: 0.9027
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 4s - loss: 0.1492 - acc: 0.9367 - val_loss: 0.2255 - val_acc: 0.9002
Epoch 59/100
Epo

Epoch 51/100
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.1827 - acc: 0.9250 - val_loss: 0.1917 - val_acc: 0.9252
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 2s - loss: 0.1598 - acc: 0.9400 - val_loss: 0.1984 - val_acc: 0.9052
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 2s - loss: 0.1627 - acc: 0.9375 - val_loss: 0.1975 - val_acc: 0.8978
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 2s - loss: 0.1619 - acc: 0.9350 - val_loss: 0.1848 - val_acc: 0.9227
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70980, saving model to best_m.h5
 - 3s - loss: 0.6475 - acc: 0.7208 - val_loss: 0.7098 - val_acc: 0.5711
Epoch 2/100
Epoch 00002: val_loss improved from 0.70980 to 0.45213, saving model to best_m.h5
 - 2s - loss: 0.4515 - acc: 0.7900 - val_loss: 0.4521 - val_acc: 0.7681
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4530 - acc: 0.8045 - val_loss: 0.5093 - val_acc: 0.7830
Epoch 4/100
Epoch 00004: val_loss improved from 0.45213 to

In [6]:
import pickle
with open('../features/cnn_4_aug1_feat_add_early.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug1_sub_add_early.csv', index=False)
# aug_1 0.216
# this one, small range and earlystop 0.214

0.214248338003
         id  is_iceberg
0  5941774d    0.004302
1  4023181e    0.991425
2  b20200e4    0.255892
3  e7f018bb    0.997949
4  4371c8c3    0.436680
